In [1]:
import pickle
import os

download_path = os.path.join("database", 'queplan_insurance.pkl')

with open(download_path, 'rb') as f:
    response = pickle.load(f)
    

In [3]:
response["Contents"]#["Key"]

[{'Key': 'queplan_insurance/',
  'LastModified': datetime.datetime(2023, 5, 7, 3, 46, 43, tzinfo=tzutc()),
  'ETag': '"d41d8cd98f00b204e9800998ecf8427e"',
  'Size': 0,
  'StorageClass': 'STANDARD'},
 {'Key': 'queplan_insurance/POL120190177.pdf',
  'LastModified': datetime.datetime(2023, 5, 16, 12, 44, 14, tzinfo=tzutc()),
  'ETag': '"613daac2466383991854806c911edfdb"',
  'Size': 34133,
  'StorageClass': 'STANDARD'},
 {'Key': 'queplan_insurance/POL320130223.pdf',
  'LastModified': datetime.datetime(2023, 5, 7, 3, 46, 58, tzinfo=tzutc()),
  'ETag': '"bee2d07ca9342a6da89d13e74094d3a8"',
  'Size': 48920,
  'StorageClass': 'STANDARD'},
 {'Key': 'queplan_insurance/POL320150503.pdf',
  'LastModified': datetime.datetime(2023, 5, 7, 3, 46, 59, tzinfo=tzutc()),
  'ETag': '"f78aa9d55be60c4024e0cbb8b2bf03c3"',
  'Size': 42520,
  'StorageClass': 'STANDARD'},
 {'Key': 'queplan_insurance/POL320180100.pdf',
  'LastModified': datetime.datetime(2023, 5, 16, 12, 44, 15, tzinfo=tzutc()),
  'ETag': '"fee1a

In [4]:
dir(response)

['__class__',
 '__class_getitem__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__ior__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__ne__',
 '__new__',
 '__or__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__ror__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'clear',
 'copy',
 'fromkeys',
 'get',
 'items',
 'keys',
 'pop',
 'popitem',
 'setdefault',
 'update',
 'values']

In [5]:
response.keys()

dict_keys(['ResponseMetadata', 'IsTruncated', 'Contents', 'Name', 'Prefix', 'MaxKeys', 'EncodingType', 'KeyCount'])

In [ ]:
{
    'ResponseMetadata': [],
    'IsTruncated': [],
    'Contents': [
        {'Key': 'queplan_insurance/',
         'LastModified': datetime.datetime(2023, 5, 7, 3, 46, 43, tzinfo=tzutc()),
         'ETag': '"d41d8cd98f00b204e9800998ecf8427e"',
         'Size': 0,
         'StorageClass': 'STANDARD'},
        {'Key': 'queplan_insurance/POL120190177.pdf',
         'LastModified': datetime.datetime(2023, 5, 16, 12, 44, 14, tzinfo=tzutc()),
         'ETag': '"613daac2466383991854806c911edfdb"',
         'Size': 34133,
         'StorageClass': 'STANDARD'},
        {'Key': 'queplan_insurance/POL320130223.pdf',
         'LastModified': datetime.datetime(2023, 5, 7, 3, 46, 58, tzinfo=tzutc()),
         'ETag': '"bee2d07ca9342a6da89d13e74094d3a8"',
         'Size': 48920,
         'StorageClass': 'STANDARD'},
        {'Key': 'queplan_insurance/POL320150503.pdf',
         'LastModified': datetime.datetime(2023, 5, 7, 3, 46, 59, tzinfo=tzutc()),
         'ETag': '"f78aa9d55be60c4024e0cbb8b2bf03c3"',
         'Size': 42520,
         'StorageClass': 'STANDARD'},
        {'Key': 'queplan_insurance/POL320180100.pdf',
         'LastModified': datetime.datetime(2023, 5, 16, 12, 44, 15, tzinfo=tzutc()),
         'ETag': '"fee1af7f2ef3fdce68e5fd607088a716"',
         'Size': 40106,
         'StorageClass': 'STANDARD'},
        {'Key': 'queplan_insurance/POL320190074.pdf',
         'LastModified': datetime.datetime(2023, 5, 16, 12, 44, 16, tzinfo=tzutc()),
         'ETag': '"16e85cd0284abb140e57cf34617dc7fd"',
         'Size': 91256,
         'StorageClass': 'STANDARD'},
        {'Key': 'queplan_insurance/POL320200071.pdf',
         'LastModified': datetime.datetime(2023, 5, 7, 3, 47, tzinfo=tzutc()),
         'ETag': '"bd1fac3ee76a83500254fb1d53c99c3b"',
         'Size': 54238,
         'StorageClass': 'STANDARD'},
        {'Key': 'queplan_insurance/POL320200214.pdf',
         'LastModified': datetime.datetime(2023, 5, 16, 12, 44, 17, tzinfo=tzutc()),
         'ETag': '"ebc1fb615600ca6a3d199cd7d633ae71"',
         'Size': 251595,
         'StorageClass': 'STANDARD'},
        {'Key': 'queplan_insurance/POL320210063.pdf',
         'LastModified': datetime.datetime(2023, 5, 16, 12, 44, 18, tzinfo=tzutc()),
         'ETag': '"d9ce8c8d8429b329a67428492b4e4e9d"',
         'Size': 13949,
         'StorageClass': 'STANDARD'},
        {'Key': 'queplan_insurance/POL320210210.pdf',
         'LastModified': datetime.datetime(2023, 5, 16, 12, 44, 19, tzinfo=tzutc()),
         'ETag': '"d59c41a7664b6c78b81e5839781731bb"',
         'Size': 17696,
         'StorageClass': 'STANDARD'}],
    'Name': [],
    'Prefix': [],
    'MaxKeys': [],
    'EncodingType': [],
    'KeyCount': []
}
